## test 2

In [1]:
import pandas as pd
import numpy as np
import random
import time
from scipy.spatial.distance import cdist

In [2]:
data = pd.read_csv("/kaggle/input/hw3-data/data.csv",header=None).values
label = pd.read_csv("/kaggle/input/hw3-data/label.csv",header=None).values.reshape(-1)

## Q1 : Compare the SSEs

In [3]:
# ======== Euclidean =========
def euclidean_distance(a, b):
    return np.linalg.norm(a - b)

# ======== 1 - cosine =========
def cosine_similarity(a, b):
    return 1 - np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# ======== 1 - jaccard =========
def jaccard_similarity(a, b):
    return 1 - np.sum(np.minimum(a, b)) / np.sum(np.maximum(a, b))

In [4]:
# ======== K_Means Class =========
def KMeans(data, k, similarity, max_iterations=500):
    #======= init get random centroid =========
    centroids = random.sample(list(data), k)
    iterations = 0
    pre_sse = 0
    
    while True:
        # ========== Assign data point to its closest centroid ==========
        clusters = [[] for i in range(k)]
        for datapoint in data:
            distances = [similarity(datapoint, centroid) for centroid in centroids]
            cluster_index = np.argmin(distances)
            clusters[cluster_index].append(datapoint)
        
        # ========== Calculate new centroid for each cluster ============
        new_centroids = []
        
        # ========== Calculate SSE ============
        sse = 0
        for i in range(k):
            cluster = clusters[i]
            if len(cluster) == 0:
                new_centroids.append(centroids[i])
                continue
            centroid = np.mean(cluster, axis=0)
            new_centroids.append(centroid)
            sse += np.sum([similarity(datapoint, centroid) for datapoint in cluster])
        
        # ============== Check for convergence ===========
        if np.allclose(new_centroids, centroids) or sse >= pre_sse or iterations >= max_iterations:
            break
        
        # ============ Update the centroids & SSE =============
        centroids = new_centroids
        pre_sse = sse
        iterations += 1
    
    return clusters, centroids, sse

In [5]:
k = len(np.unique(label))
print("k = ", k)

k =  10


In [6]:
clusters_euclidean, centroids_euclidean, sse_euclidean = KMeans(data, k, euclidean_distance)
clusters_cosine, centroids_cosine, sse_cosine = KMeans(data, k, cosine_similarity)
clusters_jaccard, centroids_jaccard, sse_jaccard = KMeans(data, k, jaccard_similarity)

In [7]:
print("Euclidean SSE =  ", sse_euclidean)
print("Cosine SSE =  ", sse_cosine)
print("Jaccard SSE =  ", sse_jaccard)

Euclidean SSE =   16345318.58205824
Cosine SSE =   2874.384098022427
Jaccard SSE =   6492.014557631396


## Q2 : Compare the accuracies

In [8]:
class KMeans:
    def __init__(self, k=10, max_iter=100, distance_metric=euclidean_distance):
        self.k = k
        self.max_iter = max_iter
        self.distance_metric = distance_metric
   
    # ============== fit for KMeans Model ===========
    def fit(self, data):
        # ==== init centroids ===
        rowNum = data.shape[0]
        self.centroids = data[np.random.choice(rowNum, self.k, replace=False)]
        
        
        for i in range(self.max_iter):
            
            clusters = []
            for _ in range(self.k):
                clusters.append([])
            
            for datapoint in data:
                #  ====== calculates distance to each centroid =========
                distances = [self.distance_metric(datapoint, centroid) for centroid in self.centroids]
                cluster = np.argmin(distances)
                 #  ====== assigns the data point to the nearest centroid's cluster ========= 
                clusters[cluster].append(datapoint)
                
            new_centroids = []
            for cluster in clusters:
                if len(cluster) == 0:
                    new_centroids.append(np.zeros(data.shape[1]))
                else:
                    new_centroids.append(np.mean(cluster, axis=0))
                    
            # ============== Check for convergence ===========
            if np.allclose(self.centroids, new_centroids):
                break
            self.centroids = new_centroids
    
    # ============== Predict class ===========
    def predict(self, data):
        distances = []
        for datapoint in data:
            datapoint_distances = []
            for centroid in self.centroids:
                distance = self.distance_metric(datapoint, centroid)
                datapoint_distances.append(distance)
            distances.append(datapoint_distances)
        distances = np.array(distances)
        return np.argmin(distances, axis=1)

In [9]:
# ========Accuracy class=========
def Accuracy(predicted, actual):
    count = 0
    total = len(label)
    for i in range(total):
        if predicted[i] == actual[i]:
            count += 1
    return (count/total)*100

In [10]:
# ========euclidean predictions=========
euclideanKM = KMeans(k=10, max_iter=100, distance_metric=euclidean_distance)
euclideanKM.fit(data)
euclidean_predictions = euclideanKM.predict(data)

# ========cosine predictions=========
cosineKM = KMeans(k=10, max_iter=100, distance_metric=cosine_similarity)
cosineKM.fit(data)
cosine_predictions = cosineKM.predict(data)

# ========jaccard  predictions=========
jaccardKM = KMeans(k=10, max_iter=100, distance_metric=jaccard_similarity)
jaccardKM.fit(data)
jaccard_predictions = jaccardKM.predict(data)

In [11]:
print("Euclidean accuracy = ",Accuracy(euclidean_predictions,label))
print("Cosine accuracy = ",Accuracy(cosine_predictions,label))
print("Jaccard accuracy = ",Accuracy(jaccard_predictions,label))

Euclidean accuracy =  18.61
Cosine accuracy =  4.7
Jaccard accuracy =  9.44


## Q3: Set up the same stop criteria, get iterations and times to converge

In [12]:
class KMeans:
    def __init__(self, k=10, max_iter=100, distance_metric=euclidean_distance, seed=99):
        self.k = k
        self.max_iter = max_iter
        self.distance_metric = distance_metric
        self.seed = seed
    
    def fit(self, data):
        
        # ===== Set the seed for NumPy's random number generator ====
        np.random.seed(self.seed)
        rowNum = data.shape[0]
        self.centroids = data[np.random.choice(rowNum, self.k, replace=False)]
        
        prev_sse = np.inf
        start_time = time.time()
        for i in range(self.max_iter):
            clusters = []
            for _ in range(self.k):
                clusters.append([])
                
            for datapoint in data:
                distances = [self.distance_metric(datapoint, centroid) for centroid in self.centroids]
                cluster = np.argmin(distances)
                clusters[cluster].append(datapoint)
            new_centroids = []
            
            for cluster in clusters:
                if len(cluster) == 0:
                    new_centroids.append(np.zeros(X.shape[1]))
                else:
                    new_centroids.append(np.mean(cluster, axis=0))
            
            
            sse = 0
            for cluster, centroid in zip(clusters, new_centroids):
                sse += np.sum((cluster - centroid) ** 2)
                
            # ==== no change in centroid position OR SSE value increases in the next iteration ====
            if np.allclose(self.centroids, new_centroids) or sse > prev_sse:
                break
                
            self.centroids = new_centroids
            prev_sse = sse
    
        end_time = time.time() 
        self.convergence_time = end_time - start_time
            
    def predict(self, data):
        distances = []
        for datapoint in data:
            datapoint_distances = []
            for centroid in self.centroids:
                distance = self.distance_metric(datapoint, centroid)
                datapoint_distances.append(distance)
            distances.append(datapoint_distances)
        distances = np.array(distances)
        return np.argmin(distances, axis=1)

In [13]:
euclideanKM = KMeans(k=10, max_iter=500, distance_metric=euclidean_distance)
euclideanKM.fit(data)

cosineKM = KMeans(k=10, max_iter=500, distance_metric=cosine_similarity)
cosineKM.fit(data)

jaccardKM = KMeans(k=10, max_iter=500, distance_metric=jaccard_similarity)
jaccardKM.fit(data)

In [14]:
print("Euclidean converged in {} iterations".format(len(euclideanKM.centroids)))
print("Euclidean take {}".format(euclideanKM.convergence_time), " to converged")
print("======================================")
print("Cosine converged in {} iterations".format(len(cosineKM.centroids)))
print("Cosine take {}".format(cosineKM.convergence_time), " to converged")
print("======================================")
print("Jaccard converged in {} iterations".format(len(jaccardKM.centroids)))
print("Jaccard take {}".format(jaccardKM.convergence_time), " to converged")

Euclidean converged in 10 iterations
Euclidean take 155.50824642181396  to converged
Cosine converged in 10 iterations
Cosine take 75.82860040664673  to converged
Jaccard converged in 10 iterations
Jaccard take 71.40520787239075  to converged


## Q4-1: Compare the SSEs : when there is no change in centroid position

In [15]:
class KMeans:
    def __init__(self, k=10, max_iter=100, distance_metric=euclidean_distance, seed=99):
        self.k = k
        self.max_iter = max_iter
        self.distance_metric = distance_metric
        self.seed = seed

    def fit(self, data):
        np.random.seed(self.seed)
        rowNum = data.shape[0]
        self.centroids = data[np.random.choice(rowNum, self.k, replace=False)]
        
        prev_centroids = None
        for i in range(self.max_iter):
            clusters = []
            for _ in range(self.k):
                clusters.append([])
                
            for datapoint in data:
                distances = cdist(datapoint.reshape(1, -1), self.centroids, metric=self.distance_metric)
                cluster = np.argmin(distances)
                clusters[cluster].append(datapoint)
          
            prev_centroids = self.centroids.copy()
            self.centroids = [np.mean(cluster, axis=0) for cluster in clusters]
            if np.allclose(prev_centroids, self.centroids):
                break

    def predict(self, data):
        distances = []
        for datapoint in data:
            datapoint_distances = []
            for centroid in self.centroids:
                distance = self.distance_metric(datapoint, centroid)
                datapoint_distances.append(distance)
            distances.append(datapoint_distances)
        distances = np.array(distances)
        return np.argmin(distances, axis=1)

In [16]:
euclideanKM = KMeans(k=10, max_iter=100, distance_metric=euclidean_distance, seed=99)
euclideanKM.fit(data)
euclidean_sse = np.sum(np.min(cdist(data, euclideanKM.centroids, 'euclidean'), axis=1)**2)


cosineKM = KMeans(k=10, max_iter=100, distance_metric=cosine_similarity, seed=99)
cosineKM.fit(data)
cosine_sse = np.sum(np.min(cdist(data, cosineKM.centroids, 'cosine'), axis=1)**2)


jaccardKM = KMeans(k=10, max_iter=100, distance_metric=jaccard_similarity, seed=99)
jaccardKM.fit(data)
jaccard_sse = np.sum(np.min(cdist(data, jaccardKM.centroids, 'jaccard'), axis=1)**2)

print("Euclidean SSE:", euclidean_sse)
print("Cosine SSE:", cosine_sse)
print("Jaccard SSE:", jaccard_sse)

Euclidean SSE: 25408545285.977436
Cosine SSE: 684.4072012471529
Jaccard SSE: 9999.69801786809


## Q4-2: Compare the SSEs : when the SSE value increases in the next iteration

In [17]:
class KMeans:
    def __init__(self, k=10, max_iter=100, distance_metric=euclidean_distance, seed=99):
        self.k = k
        self.max_iter = max_iter
        self.distance_metric = distance_metric
        self.seed = seed

    def fit(self, data):
        np.random.seed(self.seed)
        rowNum = data.shape[0]
        self.centroids = data[np.random.choice(rowNum, self.k, replace=False)]
        
        prev_centroids = None
        prev_sse = float('inf')
        
        for i in range(self.max_iter):
            clusters = []
            for _ in range(self.k):
                clusters.append([])
                
            for datapoint in data:
                distances = cdist(datapoint.reshape(1, -1), self.centroids, metric=self.distance_metric)
                cluster = np.argmin(distances)
                clusters[cluster].append(datapoint)
                
            prev_centroids = self.centroids.copy()
            self.centroids = [np.mean(cluster, axis=0) for cluster in clusters]
            sse = np.sum([np.sum(cdist(cluster, centroid.reshape(1,-1), metric=self.distance_metric)**2) for cluster, centroid in zip(clusters, self.centroids)])
            
            if sse > prev_sse:
                break
            prev_sse = sse
            
            if np.allclose(prev_centroids, self.centroids):
                break

    def predict(self, data):
        distances = []
        for datapoint in data:
            datapoint_distances = []
            for centroid in self.centroids:
                distance = self.distance_metric(datapoint, centroid)
                datapoint_distances.append(distance)
            distances.append(datapoint_distances)
        distances = np.array(distances)
        return np.argmin(distances, axis=1)

In [18]:
euclideanKM = KMeans(k=10, max_iter=100, distance_metric=euclidean_distance, seed=99)
euclideanKM.fit(data)
euclidean_sse = np.sum(np.min(cdist(data, euclideanKM.centroids, 'euclidean'), axis=1)**2)


cosineKM = KMeans(k=10, max_iter=100, distance_metric=cosine_similarity, seed=99)
cosineKM.fit(data)
cosine_sse = np.sum(np.min(cdist(data, cosineKM.centroids, 'cosine'), axis=1)**2)


jaccardKM = KMeans(k=10, max_iter=100, distance_metric=jaccard_similarity, seed=99)
jaccardKM.fit(data)
jaccard_sse = np.sum(np.min(cdist(data, jaccardKM.centroids, 'jaccard'), axis=1)**2)

print("Euclidean SSE:", euclidean_sse)
print("Cosine SSE:", cosine_sse)
print("Jaccard SSE:", jaccard_sse)

Euclidean SSE: 25408545285.977436
Cosine SSE: 684.4072012471529
Jaccard SSE: 9999.727602183244


## Q4-3: Compare the SSEs : when the maximum preset value (e.g., 100) of iteration is complete

In [19]:
class KMeans:
    def __init__(self, k=10, max_iter=100, distance_metric=euclidean_distance, seed=99):
        self.k = k
        self.max_iter = max_iter
        self.distance_metric = distance_metric
        self.seed = seed

    def fit(self, data):
        np.random.seed(self.seed)
        rowNum = data.shape[0]
        self.centroids = data[np.random.choice(rowNum, self.k, replace=False)]
        
        prev_centroids = None
        for i in range(self.max_iter):
            clusters = []
            for _ in range(self.k):
                clusters.append([])
                
            for datapoint in data:
                distances = cdist(datapoint.reshape(1, -1), self.centroids, metric=self.distance_metric)
                cluster = np.argmin(distances)
                clusters[cluster].append(datapoint)
                
            prev_centroids = self.centroids.copy()
            self.centroids = [np.mean(cluster, axis=0) for cluster in clusters]
            
            if np.allclose(prev_centroids, self.centroids):
                break
                
            elif i == self.max_iter-1:
                print(f"KMeans did not converge in {self.max_iter} iterations")
                break

    def predict(self, data):
        distances = []
        for datapoint in data:
            datapoint_distances = []
            for centroid in self.centroids:
                distance = self.distance_metric(datapoint, centroid)
                datapoint_distances.append(distance)
            distances.append(datapoint_distances)
        distances = np.array(distances)
        return np.argmin(distances, axis=1)

In [20]:
euclideanKM = KMeans(k=10, max_iter=100, distance_metric=euclidean_distance, seed=99)
euclideanKM.fit(data)
euclidean_sse = np.sum(np.min(cdist(data, euclideanKM.centroids, 'euclidean'), axis=1)**2)


cosineKM = KMeans(k=10, max_iter=100, distance_metric=cosine_similarity, seed=99)
cosineKM.fit(data)
cosine_sse = np.sum(np.min(cdist(data, cosineKM.centroids, 'cosine'), axis=1)**2)


jaccardKM = KMeans(k=10, max_iter=100, distance_metric=jaccard_similarity, seed=99)
jaccardKM.fit(data)
jaccard_sse = np.sum(np.min(cdist(data, jaccardKM.centroids, 'jaccard'), axis=1)**2)

print("Euclidean SSE:", euclidean_sse)
print("Cosine SSE:", cosine_sse)
print("Jaccard SSE:", jaccard_sse)

KMeans did not converge in 100 iterations
KMeans did not converge in 100 iterations
KMeans did not converge in 100 iterations
Euclidean SSE: 25408545285.977436
Cosine SSE: 684.4072012471529
Jaccard SSE: 9999.69801786809
